1. 사용자에게 필요한 정보 얻기

In [1]:
# Youtube Review Crawler
# 출처: https://goatlab.tistory.com/236 [GOATLAB:티스토리]

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

from urllib import parse

import pandas as pd

import time
import sys
import math
import re
import random
import os
import pprint
import platform
import datetime
import csv

is_mac = False
if (platform.platform()[:5]=="macOS"):
    is_mac = True
    print("platform is mac")

# Environmental Variable. 작업 환경에 맞게 변경
youtube_url = "https://www.youtube.com"

chromedriver_path = ""  
folder_name_to_store_data = ""

if(is_mac):
    chromedriver_path = "/usr/local/bin/chromedriver"  
    folder_name_to_store_data = "/Users/dongseokbaik/Documents/"
else:
    chromedriver_path = "C:\\Users\\User\\chromedriver.exe"  
    folder_name_to_store_data = "C:\\Test\\"
    

print("[Result] module loaded.")

# Function Definition
# 화면을 이동하여 영상 목록 출력
# 출처 : https://heytech.tistory.com/325

def parsing_youtube_search_result_item(tag):
    
    item_dict = dict()
    href_elem = tag.find('a',id='video-title')
 
    if href_elem:
        item_dict['title']=href_elem['title']
        item_dict['link']= youtube_url + href_elem['href']
    if item_dict['link'].find('/shorts/') != -1:
        return dict()
    # div  id="metadata-line"
    # ytd-badge-supported-renderer class="inline-metadata-badge style-scope ytd-video-meta-block"
    # span class="inline-metadata-item style-scope ytd-video-meta-block
    metadata_line = tag.find('div',id='metadata-line')
    meta_data=[]
    if metadata_line:
        meta_elems = metadata_line.find_all('span')
        
    for meta in meta_elems:
        meta_data.append(meta.get_text())
    
    if(len(meta_data)==2):
        item_dict['viewcount']=meta_data[0].strip()[4:]
        item_dict['date']=meta_data[1]
        
    channel_elem = tag.find('tp-yt-paper-tooltip','style-scope ytd-channel-name')
    if channel_elem:
        item_dict['channel'] = channel_elem.get_text().strip()

    return item_dict

def print_dictionary(videos):
    pp = pprint.PrettyPrinter(width=41, compact=True)
    pp.pprint(videos)

# scroll count가 0이면 무한 반복
def scroll(driver, scrollCount = 0):
    try:
        incremental = 0
        counter = 0
        if(scrollCount!=0):
            incremental = 1
            
        # 페이지 내 스크롤 높이 받아오기
        last_page_height = driver.execute_script('return document.documentElement.scrollHeight;')
#        print(f"{last_page_height} current page scroll")
        while (counter<=scrollCount) :
            # 임의의 페이지 로딩 시간 설정
            # PC환경에 따라 로딩시간 최적화를 통해 scraping 시간 단축 가능
            counter = counter + incremental
            # 페이지 최하단까지 스크롤
            driver.execute_script('window.scrollTo(0, document.documentElement.scrollHeight);')
            # 쉬는 시간이 중요하네. 스크롤링이 안되면 시간조절할 것
            pause_time = random.uniform(3,5)
            time.sleep(pause_time)

            # 페이지 내 스크롤 높이 새롭게 받아오기
            new_page_height = driver.execute_script(' return document.documentElement.scrollHeight;')
#            print(f"{new_page_height} new page scroll")
            # 스크롤을 완료한 경우(더이상 페이지 높이 변화가 없는 경우)
            if new_page_height == last_page_height:
                print("[Result] 스크롤 완료")
                break                       
            last_page_height = new_page_height
            
    except Exception as e:
        print("에러 발생: ", e)

def get_infomation_from_review_tag(tag):

    review = dict()
      
    found_tag = tag.find('a','yt-simple-endpoint style-scope ytd-comment-renderer', id='author-text')
    if found_tag:
       review['author'] = found_tag.get_text().strip()
    
    found_tag = tag.find('yt-formatted-string','published-time-text style-scope ytd-comment-renderer')
    if found_tag:
       review['date'] = found_tag.get_text().strip()
    
    found_tag = tag.find('yt-formatted-string','style-scope ytd-comment-renderer', id='content-text')
    if found_tag:
       review['comment'] = found_tag.get_text();
    
    found_tag = tag.find('span','style-scope ytd-comment-action-buttons-renderer', id = 'vote-count-middle')
    if found_tag:
        review['like'] = found_tag.get_text().strip()
        
    return review

def parsing_review(comment_item):

    review = get_infomation_from_review_tag(comment_item)
    
    comment_reply = comment_item.find('div', "style-scope ytd-comment-replies-renderer",id="expander-contents")
    review['reply'] = []
    
    if comment_reply:
        print("parsing_review....")
        review['reply'] = [ get_infomation_from_review_tag(r) for r in comment_reply.find_all('ytd-comment-renderer','style-scope ytd-comment-replies-renderer')]
            
    return review



def get_chromedriver():
    driver = None
    Options= webdriver.ChromeOptions()
    user_agent= "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36"
    Options.add_argument('user-agent=' + user_agent)
    driver= webdriver.Chrome(options=Options)

    return driver

def expand_reply(driver):
    reply_buttons = driver.find_elements(By.CSS_SELECTOR,"ytd-button-renderer#more-replies > a")
    nButton = 0
    time.sleep(2)

    for reply_button in reply_buttons:
        sleeptime = 1 # 5+nButton//5
#        print(f'tag : {reply_button.tag_name} sleep : {sleeptime}')
        reply_button.send_keys(Keys.ENTER)

        time.sleep(sleeptime)

        actions = ActionChains(driver)
        actions.click(reply_button).perform()
        nButton = nButton + 1

    return False
# False는 완료의 의미

def expand_reply_more(driver):
    nButtons = 0
    reply_more_buttons = driver.find_elements(By.CSS_SELECTOR,'ytd-continuation-item-renderer.style-scope.ytd-comment-replies-renderer>div#button>ytd-button-renderer>a>tp-yt-paper-button')

    for reply_more in reply_more_buttons:
        attribute = reply_more.get_attribute("aria-label")
        if attribute==None:
            continue
        
        if attribute =="답글 더보기":
            print(f'답글 더보기 : {reply_more.tag_name} visible {reply_more.text.strip()}')
            driver.execute_script("arguments[0].scrollIntoView(true);", reply_more);
            reply_more.send_keys(Keys.ENTER)
            nButtons = nButtons + 1
            time.sleep(2)

    return nButtons!=0
# False는 더이상 답글 더보기가 없다는 의미. 완료됨

def expand_detail(driver):
    more_buttons = driver.find_elements(By.CSS_SELECTOR,'div#comment-content>ytd-expander>tp-yt-paper-button#more')#'//*[@id="more"]')
    print(f"버튼 : {len(more_buttons)}")
    for more_button in more_buttons:
        attribute = more_button.get_attribute("hidden")
        if attribute!=None:
            continue
        if more_button.text.strip()=="자세히 보기":
            print(f'자세히 보기 : {more_button.tag_name} visible {more_button.text.strip()}')
            driver.execute_script("arguments[0].scrollIntoView(true);", more_button);
            more_button.send_keys(Keys.ENTER)
            time.sleep(5)
    return False
# False는 완료의 의미


def wait_for_webpage_loading():
    print("[Result] Waiting ....")
    time.sleep(3)

def get_output_filename(query,ext='csv'):
    today = datetime.datetime.today()
    
    noFilename = '\/:*?"<>|'
    for no in noFilename:
        query = query.replace(no,'')
    
    filename = f"{today.strftime('%Y%m%d%H%M')}_query_result[{query}].{ext}" 
    return filename

def make_content_info_row(row_number, item_content):
    row = []
    row.append(str(row_number))
    row.append(item_content['channel'])
    row.append(item_content['date'])
    row.append(item_content['link'])
    row.append(item_content['title'])
    row.append(item_content['viewcount'])
    return row

def make_review_row(row_number, item_content, item_review):
    result = []
    row = make_content_info_row(row_number,item_content)
    row.append(str(row_number))
    row.append(item_review['author'])
    row.append(item_review['date'])
    row.append(item_review['comment'])
    row.append(item_review['like'])
    
    replys = item_review['reply']
    reply_row_number = row_number
#    print(type(replys))
    for reply in replys:
        reply_row_number = reply_row_number + 1
        reply_row = make_content_info_row(reply_row_number, item_content)
        reply_row.append(str(row_number))
#        print(type(reply))
        
        reply_row.append(reply['author'])
        reply_row.append(reply['date'])
        reply_row.append(reply['comment'])
        reply_row.append(reply['like'])
        result.append(reply_row)

    result.insert(0,row)

    return result;

print("[Result] functions loaded.")

query_txt = input("youtube에서 검색할 주제 키워드 입력(예:내셔널지오그래픽): ")
print("------ input --------")
print(f"[Result] Query Text : {query_txt}")
print(f"[Result] Output filename : {folder_name_to_store_data+get_output_filename(query_txt)}")


[Result] module loaded.
[Result] functions loaded.
youtube에서 검색할 주제 키워드 입력(예:내셔널지오그래픽): 마이네임 드라마 리뷰
------ input --------
[Result] Query Text : 마이네임 드라마 리뷰
[Result] Output filename : C:\Test\202210261659_query_result[마이네임 드라마 리뷰].csv


2. 키워드 검색 / 동영상 목록 추출 / 동영상별 리뷰 추출

In [2]:
# 크롬 드라이버를 사용하여 웹 브라우저 실행 후 영상 검색

search_url = youtube_url + "/results?search_query=" + parse.quote(query_txt) + "&sp=CAM%253D"
driver = get_chromedriver()

driver.get(search_url)
driver.implicitly_wait(30)

scroll(driver)
driver.implicitly_wait(20)
# 검색된 youtube 영상의 URL 추출

soup1 = BeautifulSoup(driver.page_source, 'lxml')

driver.close()
print("[Result] Search Results have been loaded.")

video_items = []
# 검색 결과에서 동영상 리스트를 추출
for item in soup1.find_all('div', 'style-scope ytd-video-renderer', id='meta'):
    item = parsing_youtube_search_result_item(item)
    if len(item)>0:
        video_items.append(item)
print_dictionary(video_items)

print(f"[Result] {len(video_items)} items have been collected.")



[Result] 스크롤 완료
[Result] Search Results have been loaded.
[{'channel': '고몽',
  'date': '1년 전',
  'link': 'https://www.youtube.com/watch?v=YR6f7aqqD_M',
  'title': '"또! 한국이.. 넷플릭스 짱먹었네?!" 한소희의 '
           '벌크업 근육이 미쳐 날뛰는 ⟪마이네임⟫ 1-3회 '
           '소개',
  'viewcount': '394만회'},
 {'channel': '기묘한 케이지',
  'date': '1년 전',
  'link': 'https://www.youtube.com/watch?v=wLMtCyIFPx8',
  'title': '“넷플릭스.. 한국 드라마가 또 찢었다!?” '
           '10kg 벌크업한 한소희의 갬성액숀느와르 '
           '《마이네임》',
  'viewcount': '287만회'},
 {'channel': '리얼무비',
  'date': '11개월 전',
  'link': 'https://www.youtube.com/watch?v=RJixwgdw4nI',
  'title': '넷플릭스 《마이네임》 처음부터 끝까지 35분만에 '
           '몰아보기!! [ep1-ep8]',
  'viewcount': '265만회'},
 {'channel': '어바웃타임',
  'date': '1년 전',
  'link': 'https://www.youtube.com/watch?v=8OyhXpwPK2w',
  'title': '천재적인 싸움재능을 가진 조폭 딸 잘못 건드렸다가 '
           '모두 털려버린 양아치 일진들의 최후',
  'viewcount': '212만회'},
 {'channel': '티비요정',
  'date': '1년 전',
  'link': 'https://www.youtube.com/watch?v=oDFa6mOCID0',
  'title': '전

In [3]:
df_video_items = pd.DataFrame(video_items)
print(df_video_items.head())
df_video_items.to_csv(folder_name_to_store_data+get_output_filename(query_txt), index=False, encoding="utf-8-sig", quoting=csv.QUOTE_ALL)
del df_video_items

                                               title  \
0  "또! 한국이.. 넷플릭스 짱먹었네?!" 한소희의 벌크업 근육이 미쳐 날뛰는 ⟪마이...   
1  “넷플릭스.. 한국 드라마가 또 찢었다!?” 10kg 벌크업한 한소희의 갬성액숀느와...   
2        넷플릭스 《마이네임》 처음부터 끝까지 35분만에 몰아보기!! [ep1-ep8]   
3    천재적인 싸움재능을 가진 조폭 딸 잘못 건드렸다가 모두 털려버린 양아치 일진들의 최후   
4  전세계 넷플릭스 1위 하려고 만든!! 한소희, 안보현 마이네임 마지막화 (결말포함)...   

                                          link viewcount    date  channel  
0  https://www.youtube.com/watch?v=YR6f7aqqD_M     394만회    1년 전       고몽  
1  https://www.youtube.com/watch?v=wLMtCyIFPx8     287만회    1년 전  기묘한 케이지  
2  https://www.youtube.com/watch?v=RJixwgdw4nI     265만회  11개월 전     리얼무비  
3  https://www.youtube.com/watch?v=8OyhXpwPK2w     212만회    1년 전    어바웃타임  
4  https://www.youtube.com/watch?v=oDFa6mOCID0     159만회    1년 전     티비요정  


In [ ]:
soap1= None

# 동영상별 리뷰 추출
f = open("C:\\Test\\"+get_output_filename("tv"), 'w', encoding='utf-8', newline='')
wr = csv.writer(f,quotechar='"', quoting=csv.QUOTE_ALL)
n_csv_item = 1

print("C:\\Test\\"+get_output_filename("tv"))

for video_item in video_items:
    print("check")
    #driver2 = get_chromedriver()
    driver2 = webdriver.Chrome(executable_path="C:\\Users\\User\\chromedriver.exe")
    
    driver2.set_window_size(800, 800)
    driver2.get(video_item['link'])
    #driver2.get(test_link)
    driver2.implicitly_wait(10)

    wait_for_webpage_loading()

    # 리뷰를 모두 얻기 위해 끝까지 스크롤링
    scroll(driver2)

    #리뷰에 자세히 보기가 있다면 모두 클릭

    # 대댓글 펼치기
    wait_for_webpage_loading()
    expand_reply(driver2)


    # 답글 더보기 : 답글 더보기가 없을 때까지 반복
    nCount = 0
    while expand_reply_more(driver2):
        #print("check")
        nCount = nCount + 1
        wait_for_webpage_loading()
        print(f"Click reply more {nCount}")


    # 자세히 보기
    wait_for_webpage_loading()
    expand_detail(driver2)

    wait_for_webpage_loading()
    soup2 = BeautifulSoup(driver2.page_source, 'lxml')

    reviews = [ parsing_review(comment_item) for comment_item in soup2.find_all('ytd-comment-thread-renderer','style-scope ytd-item-section-renderer')]
    print_dictionary(reviews)

    
    for comment_item in soup2.find_all('ytd-comment-thread-renderer','style-scope ytd-item-section-renderer'):
        rows = make_review_row(n_csv_item, video_item, parsing_review(comment_item))
        wr.writerows(rows)
        n_csv_item = n_csv_item + len(rows)
        f.flush()
        
    

f.close()        
driver2.quit()

print("[Result] Finished!!!!!!!!!!!!!")

C:\Test\202210261701_query_result[tv].csv
check


C:\Users\User\AppData\Local\Temp\ipykernel_5196\2473873644.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver2 = webdriver.Chrome(executable_path="C:\\Users\\User\\chromedriver.exe")


[Result] Waiting ....


In [29]:
f.close()